<a href="https://colab.research.google.com/github/pko89403/Recsys_test/blob/master/item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import random
from collections import Counter
import pandas as pd

In [0]:
class SGNS(nn.Module): # Skip-Gram Model with Negative Sampling
  def __init__(self, total_items, embedding_dimensions):
    super(SGNS, self).__init__()
  
  def forward(self, pos_v, neg_v):
    pass

In [0]:
!pwd

/content


In [0]:
!ls /content/

sample_data


In [0]:
DATASET_PATH = "/content/drive/My Drive/data/ml-1m/ratings.dat"

In [0]:
with open(DATASET_PATH, 'r') as f:
  print(f.readline())

1::1193::5::978300760

